# Reslice CT in multiples of Slice Spacing

### This is a simple algorithm to increase (only) the Spacing Between Slices (0018, 0088) of CT scans.
### It assumes the Slice Thickness is equal to Spacing Between Slices.
### Otherwise, it calculates only an approximation of the real result, which may also be useful.

## 1. Import libraries

In [1]:
import pydicom
import cv2
import numpy as np
from glob import glob
import os

## 2. Load a series and sort based on slice position 

In [2]:
_e = "study\\series"
slices = [pydicom.read_file(_e + '\\' + s) for s in os.listdir(_e)]
slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

In [3]:
print("Number of slices: " + str(len(slices)))

Number of slices: 168


## 3. Check if Spacing Between Slices matches the distance between each adjacent slice 

In [4]:
for i in range(1, len(slices)):
    distance_between_adjacent_slices = slices[i].ImagePositionPatient[2] - \
                                        slices[i-1].ImagePositionPatient[2]
    
    if slices[i].SpacingBetweenSlices == distance_between_adjacent_slices:
        print("Slice #" + str(i) + " pass. SliceSpacing: " + \
              str(slices[i].SpacingBetweenSlices) + "mm Distance: " + \
              str(distance_between_adjacent_slices) + "mm.")
    else:
        print("Inconsistency found in slice #" + str(i) + ". SliceSpacing: " + \
              str(slices[i].SpacingBetweenSlices) + "mm Distance: " + \
              str(distance_between_adjacent_slices) + "mm.")
        

Slice #1 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #2 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #3 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #4 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #5 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #6 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #7 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #8 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #9 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #10 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #11 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #12 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #13 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #14 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #15 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #16 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #17 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #18 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Slice #19 pass. SliceSpacing: 1.0mm Distance: 1.0mm.
Sl

## 4. Print some possible target Spacing Between Slices
### These are multiples of the original Spacing Between Slices

In [5]:
for c in range(2, 16):
    print("Multiplying " + str(slices[1].SpacingBetweenSlices) + "mm by " + \
          str(c) + ": " + str(slices[1].SpacingBetweenSlices * c))


Multiplying 1.0mm by 2: 2.0
Multiplying 1.0mm by 3: 3.0
Multiplying 1.0mm by 4: 4.0
Multiplying 1.0mm by 5: 5.0
Multiplying 1.0mm by 6: 6.0
Multiplying 1.0mm by 7: 7.0
Multiplying 1.0mm by 8: 8.0
Multiplying 1.0mm by 9: 9.0
Multiplying 1.0mm by 10: 10.0
Multiplying 1.0mm by 11: 11.0
Multiplying 1.0mm by 12: 12.0
Multiplying 1.0mm by 13: 13.0
Multiplying 1.0mm by 14: 14.0
Multiplying 1.0mm by 15: 15.0


## 5. Choose a target SpacingBetweenSlices. It has to be a multiple of the original SpacingBetweenSlices.

In [6]:
target_SpacingBetweenSlices = 2.0

In [7]:
count = 0

slices = [pydicom.read_file(_e + '\\' + s) for s in os.listdir(_e)]
slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

target_SpacingBetweenSlices = int(target_SpacingBetweenSlices / slices[1].SpacingBetweenSlices)

slab = np.empty((slices[0].Rows, slices[0].Columns, target_SpacingBetweenSlices))

meio = target_SpacingBetweenSlices // 2

for i in range(len(slices)):
    
    slab[..., count] = slices[i].pixel_array
    
    count += 1
    if count == target_SpacingBetweenSlices:
        count = 0
        media = np.mean(slab, axis=2).astype("uint16")
        slices[i-meio].PixelData = media.tobytes()
        
        Thickness = target_SpacingBetweenSlices * slices[i].SpacingBetweenSlices
        
        slices[i-meio].SliceThickness = Thickness
        slices[i-meio].SpacingBetweenSlices = Thickness
        
        os.makedirs("reformated\\" + str(Thickness) + "mm\\" + str(slices[i-meio].StudyInstanceUID) + \
                    "\\", exist_ok=True)
        pydicom.dcmwrite("reformated\\" + str(Thickness) + "mm\\" + str(slices[i-meio].StudyInstanceUID) + \
                         "\\" + str(slices[i-meio].SOPInstanceUID), slices[i-meio]) 

del(slices)